In [ ]:
import numpy as np
import pandas as pd
import mne 
import os 
from natsort import natsorted
import re
import gc
from scipy.signal import hilbert
from sklearn.decomposition import  PCA

In [ ]:
# determine filter parameters   (Hamilton et al., 2021)
freqs = np.round(np.logspace(*np.log10([70, 136]), num=8),2) 

In [ ]:
pca = PCA(n_components=2)

In [ ]:
fname = 'epochsfile.fif'
epochs = mne.read_epochs(fname, proj=True, preload=True, verbose=None) 
# get HFa response for each condition for each electrode
electrodes = epochs.info['ch_names']
infoHGB = epochs.metadata
nconds,nchans,ntimes = epochs.get_data().shape 
HGB_all = np.empty((nconds,nchans,ntimes)) 

epo_data = epochs.get_data()
for num in range(nconds):
    for numi in range(nchans):
        hlbr = np.empty((len(freqs),ntimes))
        for i, (nombre,freq) in enumerate(zip(range(len(freqs)),freqs)):
            signal = mne.filter.filter_data(epo_data[num,numi,:], sfreq=epochs.info['sfreq'],
                                                l_freq=freq-1, h_freq=freq+1,  
                                                method='iir', verbose=False,n_jobs=-1,) #fir
            signal = (signal - np.mean(signal)) / np.std(signal)
            signal = np.abs(hilbert(signal))
            hlbr[nombre]= signal
        unit_gamma = pca.fit_transform(hlbr.T)[:,0]
        HGB_all[num,numi,:] = unit_gamma
# recreate epochs structure with HFa data
print('done!')
ch_types = ['seeg']*nchans
info = mne.create_info(electrodes, ch_types=ch_types, sfreq=epochs.info['sfreq'])

HGB_epochs = mne.EpochsArray(HGB_all, info,tmin=-2)
# ad dataframe containing conditions
HGB_epochs.metadata = epochs.metadata
HGB_epochs = HGB_epochs.resample(256.0)
final_path = '\\epochsHFa\\'
if not os.path.exists(final_path):
    os.makedirs(final_path)
HGB_epochs.save(final_path+'_HGB-epo.fif', overwrite=True)
del epochs, epo_data
gc.collect()